Dependencias y configuración

In [2]:
pip install ijson pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 5.2 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/Dataton/JsonsEntrenamiento"
MODELS_DIR = "/content/drive/MyDrive/Dataton/Modelos"
OUTPUT_DIR = "/content/drive/MyDrive/Dataton/Salidas"
import os
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("DATA_DIR   =", DATA_DIR)
print("MODELS_DIR =", MODELS_DIR)
print("OUTPUT_DIR =", OUTPUT_DIR)

Mounted at /content/drive
DATA_DIR   = /content/drive/MyDrive/Dataton/JsonsEntrenamiento
MODELS_DIR = /content/drive/MyDrive/Dataton/Modelos
OUTPUT_DIR = /content/drive/MyDrive/Dataton/Salidas


In [3]:
import json
import pandas as pd
import numpy as np
from decimal import Decimal
from sklearn.ensemble import IsolationForest
from joblib import dump, load
import matplotlib.pyplot as plt
import seaborn as sns



Preprocesamiento (limpieza y preparación de los datos)

In [4]:
def num_from_valor(v):
    if isinstance(v, dict):
        if "$numberDecimal" in v:
            return float(v["$numberDecimal"])
        if "valor" in v:
            return num_from_valor(v["valor"])
    try:
        return float(v)
    except:
        return 0.0

def suma_lista(lista, campos_posibles):
    total = 0.0
    if not isinstance(lista, list):
        return 0.0
    for item in lista:
        if not isinstance(item, dict):
            continue
        for campo in campos_posibles:
            if campo in item:
                total += float(num_from_valor(item[campo]))
                break
    return total


In [5]:
def extraer_campos_s1(obj):
    metadata = obj.get("metadata", {})
    declaracion = obj.get("declaracion", {})
    sitp = declaracion.get("situacionPatrimonial", {})
    fecha_act = metadata.get("actualizacion")
    try:
        anio = pd.to_datetime(fecha_act).year
    except:
        anio = None

    institucion = metadata.get("institucion", "")
    tipo_declaracion = metadata.get("tipo", "")
    datos_gen = sitp.get("datosGenerales", {})
    nombre = datos_gen.get("nombre", "")
    primer_ap = datos_gen.get("primerApellido", "")
    segundo_ap = datos_gen.get("segundoApellido", "")

    id_persona = f"{nombre} {primer_ap} {segundo_ap}".strip()

    ingresos = sitp.get("ingresos", {})
    ingreso_neto = num_from_valor(
        ingresos.get("ingresoAnualNetoDeclarante", {}).get("valor", 0)
    )
    total_ingresos_anuales = num_from_valor(
        ingresos.get("totalIngresosAnualesNetos", {}).get("valor", 0)
    )

    if total_ingresos_anuales < 0:
        total_ingresos_anuales = 0

    if ingreso_neto < 0:
        ingreso_neto = 0

    if total_ingresos_anuales == 0 and ingreso_neto > 0:
        total_ingresos_anuales = ingreso_neto

    bienes = sitp.get("bienesInmuebles", {}).get("bienInmueble", [])
    bienes_val = suma_lista(bienes, ["valorAdquisicion"])

    muebles = sitp.get("bienesMuebles", {}).get("bienMueble", [])
    muebles_val = suma_lista(muebles, ["valorAdquisicion"])

    vehiculos = sitp.get("vehiculos", {}).get("vehiculo", [])
    vehiculos_val = suma_lista(vehiculos, ["valorAdquisicion"])

    inversiones = sitp.get("inversiones", {}).get("inversion", [])
    inversiones_val = suma_lista(inversiones, ["saldo", "montoOriginal"])

    activos_totales = bienes_val + muebles_val + inversiones_val + vehiculos_val

    adeudos = sitp.get("adeudos", {}).get("adeudo", [])
    pasivos_totales = suma_lista(adeudos, ["montoOriginal"])

    #Banderas
    interes = declaracion.get("interes", {})
    fideicomisos = interes.get("fideicomisos", {})
    tiene_fideicomisos = int(not fideicomisos.get("ninguno", True))
    num_fideicomisos = 0
    if isinstance(fideicomisos.get("fideicomiso"), list):
        num_fideicomisos = len(fideicomisos["fideicomiso"])

    tiene_adeudos = int(len(adeudos) > 0)
    tiene_inversiones = int(len(inversiones) > 0)

    return {
        "id_persona": id_persona,
        "anio": anio,
        "institucion": institucion,
        "tipo_declaracion": tipo_declaracion,
        "ingreso_neto": ingreso_neto,
        "total_ingresos_anuales": total_ingresos_anuales,
        "activos_totales": activos_totales,
        "pasivos_totales": pasivos_totales,
        "tiene_fideicomisos": tiene_fideicomisos,
        "num_fideicomisos": num_fideicomisos,
        "tiene_adeudos": tiene_adeudos,
        "tiene_inversiones": tiene_inversiones,
    }


In [6]:
def procesar_json(path_json):
    print("Leyendo JSON:", path_json)

    with open(path_json, "r", encoding="utf-8") as f:
        data = json.load(f)

    registros = []
    for obj in data:
        try:
            reg = extraer_campos_s1(obj)
            if reg["anio"] is not None and reg["id_persona"]:
                registros.append(reg)
        except Exception as e:
            print("  Error en registro:", e)

    df = pd.DataFrame(registros)
    print("Registros extraídos:", df.shape)
    return df


In [7]:
def construir_tramos(df_panel):
    df = df_panel.sort_values(["id_persona", "anio"]).copy()

    df["anio_prev"] = df.groupby("id_persona")["anio"].shift(1)

    df["patrimonio_neto"] = df["activos_totales"] - df["pasivos_totales"]
    df["patrimonio_prev"] = df.groupby("id_persona")["patrimonio_neto"].shift(1)

    df["delta_patrimonio"] = df["patrimonio_neto"] - df["patrimonio_prev"]

    df["ingresos_prev"] = df.groupby("id_persona")["total_ingresos_anuales"].shift(1)
    df["ingresos_acumulados"] = df["total_ingresos_anuales"] + df["ingresos_prev"].fillna(0)

    df["residuo"] = df["delta_patrimonio"] - df["ingresos_acumulados"]

    df_tramos = df.dropna(subset=["anio_prev"]).copy()

    print("Tramos construidos:", df_tramos.shape)
    return df_tramos


In [8]:
TRAMOS_TRAIN_CSV = f"{OUTPUT_DIR}/panel_tramos_entrenamiento.csv"

def generar_tramos_entrenamiento(carpeta_jsons, salida_csv):
    dfs_tramos = []

    for nombre in os.listdir(carpeta_jsons):
        if not nombre.endswith(".json"):
            continue
        path_json = os.path.join(carpeta_jsons, nombre)
        print("\nProcesando JSON de entrenamiento:", path_json)

        df_panel = procesar_json(path_json)
        if df_panel.empty:
            print("sin registros")
            continue

        df_tramos = construir_tramos(df_panel)
        if df_tramos.empty:
            print("sin tramos")
            continue

        dfs_tramos.append(df_tramos)

    if not dfs_tramos:
        print("No se generó ningún tramo de entrenamiento.")
        return

    df_all = pd.concat(dfs_tramos, ignore_index=True)
    df_all.to_csv(salida_csv, index=False)
    print("\nTramos de entrenamiento guardados en:", salida_csv)
    print("Final:", df_all.shape)

generar_tramos_entrenamiento(DATA_DIR, TRAMOS_TRAIN_CSV)



Procesando JSON de entrenamiento: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoBanxico.json
Leyendo JSON: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoBanxico.json
Registros extraídos: (8705, 12)
Tramos construidos: (4266, 19)

Procesando JSON de entrenamiento: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoEdomex.json
Leyendo JSON: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoEdomex.json
Registros extraídos: (135864, 12)
Tramos construidos: (95395, 19)

Procesando JSON de entrenamiento: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoSinaloa.json
Leyendo JSON: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoSinaloa.json
Registros extraídos: (1252, 12)
Tramos construidos: (731, 19)

Procesando JSON de entrenamiento: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoTlaxcala.json
Leyendo JSON: /content/drive/MyDrive/Dataton/JsonsEntrenamiento/completoTlaxcala.json
Registros extraídos: (76819, 12)
Tramos const

Entrenamiento del modelo

In [10]:

FEATURES = [
    "delta_patrimonio",
    "ingresos_acumulados",
    "residuo",
    "patrimonio_neto",
    "patrimonio_prev",
    "pasivos_totales",
    "tiene_fideicomisos",
    "num_fideicomisos",
    "tiene_adeudos",
    "tiene_inversiones"
]

print("FEATURES definidas correctamente:", FEATURES)


FEATURES definidas correctamente: ['delta_patrimonio', 'ingresos_acumulados', 'residuo', 'patrimonio_neto', 'patrimonio_prev', 'pasivos_totales', 'tiene_fideicomisos', 'num_fideicomisos', 'tiene_adeudos', 'tiene_inversiones']


In [11]:
MODELO_PATH = f"{MODELS_DIR}/iso_patrimonial.joblib"

def entrenar_modelo_tramos(path_csv_tramos, path_modelo):
    print("Cargando tramos de entrenamiento:", path_csv_tramos)
    df = pd.read_csv(path_csv_tramos)

    X = df[FEATURES].fillna(0)

    print("Entrenando Isolation Forest...")
    iso = IsolationForest(
        n_estimators=200,
        contamination=0.02,
        random_state=42
    )
    iso.fit(X)

    dump(iso, path_modelo)
    print("Modelo guardado en:", path_modelo)
    print("\nCalculando anomalías en los datos de entrenamiento...")
    df["anomalia_score"] = iso.decision_function(X)
    pred = iso.predict(X)       # 1 = normal, -1 = anómalo
    df["es_anomalo"] = pred
    anom = df[df["es_anomalo"] == -1].copy()
    print(f"Total anomalías en entrenamiento: {len(anom)} de {len(df)} registros")
    print("\nEjemplo de anomalías (primeros 10):")
    display(anom.head(10))

    anom_path = f"{OUTPUT_DIR}/anomalos_entrenamiento.csv"
    anom.to_csv(anom_path, index=False)
    print("Anómalos de entrenamiento guardados en:", anom_path)

    return iso, anom

iso_model, anom_train = entrenar_modelo_tramos(TRAMOS_TRAIN_CSV, MODELO_PATH)


Cargando tramos de entrenamiento: /content/drive/MyDrive/Dataton/Salidas/panel_tramos_entrenamiento.csv
Entrenando Isolation Forest...
Modelo guardado en: /content/drive/MyDrive/Dataton/Modelos/iso_patrimonial.joblib

Calculando anomalías en los datos de entrenamiento...
Total anomalías en entrenamiento: 8202 de 410080 registros

Ejemplo de anomalías (primeros 10):


,id_persona,anio,institucion,tipo_declaracion,ingreso_neto,total_ingresos_anuales,activos_totales,pasivos_totales,tiene_fideicomisos,num_fideicomisos,...,tiene_inversiones,anio_prev,patrimonio_neto,patrimonio_prev,delta_patrimonio,ingresos_prev,ingresos_acumulados,residuo,anomalia_score,es_anomalo
3,ARMANDO CARBALLO ITURBIDE,2025,Banco de México,MODIFICACIÓN,4350895.0,4350895.0,0.0,11261300.0,0,0,...,1,2024.0,-11261300.0,-11097391.0,-163909.0,4233232.0,8584127.0,-8748036.0,-0.117729,-1
22,Abraham Isaac Lara González,2025,Banco de México,MODIFICACIÓN,1777084.0,1777084.0,2699900.0,4986406.0,0,0,...,1,2024.0,-2286506.0,-1122138.0,-1164368.0,1186816.0,2963900.0,-4128268.0,-0.097728,-1
24,Abraham Limas Romero,2025,Banco de México,MODIFICACIÓN,677529.0,677529.0,295000.0,1513961.0,0,0,...,1,2024.0,-1218961.0,-253235.0,-965726.0,413279.0,1090808.0,-2056534.0,-0.039612,-1
25,Abraham Ramírez Sánchez,2025,Banco de México,MODIFICACIÓN,913507.0,913507.0,0.0,1855294.0,0,0,...,1,2024.0,-1855294.0,-1436722.0,-418572.0,912702.0,1826209.0,-2244781.0,-0.058340,-1
27,Abril Elizabeth Chávez García,2025,Banco de México,MODIFICACIÓN,2190594.0,2190594.0,4304900.0,3846799.0,0,0,...,1,2024.0,458101.0,-1625797.0,2083898.0,1808143.0,3998737.0,-1914839.0,-0.110514,-1
33,Addy Patricia Romero Osalde,2025,Banco de México,MODIFICACIÓN,4994506.0,4994506.0,10114317.0,7933866.0,0,0,...,1,2024.0,2180451.0,-55703.0,2236154.0,2080957.0,7075463.0,-4839309.0,-0.121988,-1
35,Adolfo Gutiérrez Chávez,2025,Banco de México,MODIFICACIÓN,11742644.0,11742644.0,739900.0,4881308.0,0,0,...,1,2024.0,-4141408.0,-5705528.0,1564120.0,4499379.0,16242023.0,-14677903.0,-0.141502,-1
37,Adolfo Héctor Pulido Manzo,2025,Banco de México,MODIFICACIÓN,1112880.0,1112880.0,1920900.0,2941891.0,0,0,...,1,2024.0,-1020991.0,502505.0,-1523496.0,1074743.0,2187623.0,-3711119.0,-0.070913,-1
43,Adrian Quevedo Ortiz,2025,Banco de México,MODIFICACIÓN,1353708.0,1353708.0,274194.0,1947678.0,0,0,...,1,2024.0,-1673484.0,-2011106.0,337622.0,1269119.0,2622827.0,-2285205.0,-0.057609,-1
53,Adriana Galicia Acosta,2025,Banco de México,MODIFICACIÓN,1387604.0,1387604.0,1236000.0,3965120.0,0,0,...,1,2024.0,-2729120.0,-1407534.0,-1321586.0,1329581.0,2717185.0,-4038771.0,-0.098049,-1


Anómalos de entrenamiento guardados en: /content/drive/MyDrive/Dataton/Salidas/anomalos_entrenamiento.csv


Post procesamiento (Interpretación de los datos)

In [12]:
def pipeline_json_a_anomalias(json_path, model_path, salida_tramos_json, salida_personas_json=None):

    df_panel = procesar_json(json_path)
    if df_panel.empty:
        print("Panel vacío; no se pudo extraer información.")
        return None

    df_tramos = construir_tramos(df_panel)
    if df_tramos.empty:
        print("No hay tramos (solo una declaración por persona).")
        return None

    iso = load(model_path)

    X = df_tramos[FEATURES].fillna(0)

    df_tramos["anomalia_score"] = iso.decision_function(X)
    pred = iso.predict(X)
    df_tramos["es_anomalo"] = pred

    df_tramos.to_json(salida_tramos_json, orient="records", force_ascii=False, indent=2)
    print("Tramos (con es_anomalo) guardados en:", salida_tramos_json)

   #Lo resumimos?
    if salida_personas_json is not None:
        df_anom = df_tramos[df_tramos["es_anomalo"] == -1].copy()
        if df_anom.empty:
            print("No hay tramos anómalos; no se genera resumen por persona.")
        else:
            resumen_personas = (
                df_anom
                .groupby("id_persona")
                .agg({
                    "institucion": lambda x: sorted(set(x)),
                    "anio_prev": "min",
                    "anio": "max",
                    "es_anomalo": "count",
                    "anomalia_score": "mean",
                    "delta_patrimonio": "sum",
                    "ingresos_acumulados": "sum"
                })
                .reset_index()
                .rename(columns={
                    "es_anomalo": "num_tramos_anomalos",
                    "anio_prev": "primer_anio_observado",
                    "anio": "ultimo_anio_observado",
                    "anomalia_score": "score_promedio"
                })
            )
            resumen_personas.to_json(salida_personas_json, orient="records", force_ascii=False, indent=2)
            print("Resumen de servidores anómalos guardado en:", salida_personas_json)

    return df_tramos


Aplicación del modelo entrenado

In [13]:
JSON_NUEVO = "/content/drive/MyDrive/Dataton/JsonNuevos/completoEdomex.json"

SALIDA_TRAMOS_JSON = f"{OUTPUT_DIR}/anomalias_tramos.json"
SALIDA_PERSONAS_JSON = f"{OUTPUT_DIR}/servidores_publicos_anomalos.json"

df_resultado = pipeline_json_a_anomalias(
    json_path=JSON_NUEVO,
    model_path=MODELO_PATH,
    salida_tramos_json=SALIDA_TRAMOS_JSON,
    salida_personas_json=SALIDA_PERSONAS_JSON
)

if df_resultado is not None:
    display(df_resultado.head())


Leyendo JSON: /content/drive/MyDrive/Dataton/JsonNuevos/completoEdomex.json
Registros extraídos: (135864, 12)
Tramos construidos: (95395, 19)
Tramos (con es_anomalo) guardados en: /content/drive/MyDrive/Dataton/Salidas/anomalias_tramos.json
Resumen de servidores anómalos guardado en: /content/drive/MyDrive/Dataton/Salidas/servidores_publicos_anomalos.json


,id_persona,anio,institucion,tipo_declaracion,ingreso_neto,total_ingresos_anuales,activos_totales,pasivos_totales,tiene_fideicomisos,num_fideicomisos,...,tiene_inversiones,anio_prev,patrimonio_neto,patrimonio_prev,delta_patrimonio,ingresos_prev,ingresos_acumulados,residuo,anomalia_score,es_anomalo
50211,AARON AGUILAR FLORES,2022,Universidad Autónoma del Estado de México,MODIFICACIÓN,113253808.0,113253808.0,0.0,0.0,0,0,...,0,2021.0,0.0,0.0,0.0,0.0,113253808.0,-113253808.0,-0.009278,-1
65587,AARON AGUILAR FLORES,2023,Universidad Autónoma del Estado de México,MODIFICACIÓN,127712.0,127712.0,0.0,0.0,0,0,...,0,2022.0,0.0,0.0,0.0,113253808.0,113381520.0,-113381520.0,-0.009278,-1
102515,AARON AGUILAR FLORES,2024,Universidad Autónoma del Estado de México,MODIFICACIÓN,128521.0,128521.0,0.0,0.0,0,0,...,0,2023.0,0.0,0.0,0.0,127712.0,256233.0,-256233.0,0.340268,1
131068,AARON AGUILAR FLORES,2025,Universidad Autónoma del Estado de México,MODIFICACIÓN,147650.0,147650.0,0.0,0.0,0,0,...,0,2024.0,0.0,0.0,0.0,128521.0,276171.0,-276171.0,0.339919,1
50455,AARON AGUILAR MONTIEL,2022,Universidad Autónoma del Estado de México,MODIFICACIÓN,80511.0,80511.0,0.0,0.0,0,0,...,0,2021.0,0.0,0.0,0.0,0.0,80511.0,-80511.0,0.337802,1
